In [1]:
# integrate all datasets
Sys.setenv(RETICULATE_PYTHON = "/data1/lesliec/tyler/utils/miniforge3/envs/multiome/bin/python")
setwd("~/0-workspace/CCR7_DC/oral-tolerance-integrate/")

suppressPackageStartupMessages({
    library(curl)
    library(Seurat)
    library(anndata)
    library(Rmagic)
    library(Matrix)
    library(dplyr)
    library(ggplot2)
    library(ggrepel)
    library(cowplot)
    library(ComplexHeatmap)
    library(circlize)
    library(anndata)
    library(rasterpdf)
})
reticulate::py_require("anndata")

set.seed(1)
options(future.globals.maxSize = Inf)

source("utils.R")


In [2]:
load("../MLN_RORgt_MHCII_multiome/palette.RData")
pal <- list(annotation = rna.pal)
pal$Clusters = c(
    "#00bf00", "#489de8", "#d40663", "#f8c72f", "#077315",
    "#785cd4", "#e67109", "#0eefff", "#f081e6", "#260691",
    "#49709c", "#9e7d3f", "#bd537a", "#4e225c", "#f202ed",
    "#fec55f", "#062e0b", "#9c6fa8", "#078d94", "#5c1a1a",
    "#827c68", "#aebeff", "#9c2903", "#ffc5af", "#4f5715",
    "#0249f0", "#f43525", "#0077ff", "#7f227e", "#dfddff",
    "#7e85d7", "#fff64f", "#5fed0e", "#543018", "#f31220"
)
pal$Clusters_long = c(
    "#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
    "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
    "#5A0007", "#809693", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
    "#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
    "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
    "#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
    "#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
    "#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
    "#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
    "#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
    "#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
    "#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
    "#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C")


In [3]:
# 3. plots ####
pref.i <- "integrate-ILC3-TC-v5/"; dir.create(pref.i)
reduction <- "cca"; pref <- paste0(pref.i, reduction, "/"); dir.create(pref)
pref.p <- paste0("plots/", pref); dir.create(pref.p, recursive = T)
sro.i <- readRDS(paste0(pref, "SRO.rds"))

Warning message in dir.create(pref.i):
“'integrate-ILC3-TC-v5' already exists”
Warning message in dir.create(pref):
“'integrate-ILC3-TC-v5/cca' already exists”
Warning message in dir.create(pref.p, recursive = T):
“'plots/integrate-ILC3-TC-v5/cca' already exists”


# group plots

## sample (for Gardner)

In [4]:
group.name <- 'sample'
groups <- sort(unique(sro.i@meta.data[[group.name]]))
groups

[1] "adult_LN"                 "adult_mLN"               
 [3] "adult_spleen"             "colonic"                 
 [5] "ctrl"                     "early_LN"                
 [7] "early_mLN"                "early_spleen"            
 [9] "hCD2_neg"                 "hCD2_pos"                
[11] "hepatic"                  "kedmi"                   
[13] "lyu"                      "mediastinal"             
[15] "MLN_RORgt_MHCII_multiome" "mut"                     
[17] "para-aortic"              "Peyers patches"          
[19] "salivary gland"           "skin"                    
[21] "small intestine"          "wang"

In [5]:
sources <- c(
    "MLN_RORgt_MHCII_multiome", "TC_all_LN", "Colonna",
    "Kedmi", "Lyu", "Wang",
    "Littman", "Gardner.A", "Gardner.E"
)

pl <- list()
for (i in 1:length(sources)){
    source.curr <- sources[i]
    p.curr <- plot.clusters.highlight.one(
        SRO = sro.i,
        idx1 = colnames(sro.i), idx2 = sro.i$orig.ident == source.curr,
        groups = get.named.vector.sro(sro.i, group.name),
        clusters.col = source.curr,
        labels = F, label.size = 5,
        col = pal[[group.name]], pref.C = F) + guides(colour = guide_legend(override.aes = list(size=5), ncol = 1))
    pl[[i]] <- p.curr
}

ncols <- 3
nrows <- ceiling(length(pl)/ncols)

pdf(file = paste0(pref.p, "UMAP-", group.name, ".pdf"), width = 10*ncols, height = 6*nrows)
plot_grid(plotlist = pl, ncol = ncols, align = 'hv')
dev.off()

pdf 
  2

## paper annot

### update Colonna annot

In [6]:
res0.4.to.anno <- c('Colonna C7' = 'RORgt DC I', 'Colonna C5' = 'RORgt DC II', 'Colonna C9' = 'RORgt DC III', 'Colonna C10' = 'RORgt DC IV', 'Colonna C0' = 'ILC3')

In [7]:
sro.i@meta.data[sro.i@meta.data$orig.ident == 'Colonna',]$paper.annot <- res0.4.to.anno[sro.i@meta.data[sro.i@meta.data$orig.ident == 'Colonna',]$Cluster.prev]

## Cluster.prev

In [8]:
group.name <- 'Cluster.prev'
groups <- sort(unique(sro.i@meta.data[[group.name]]))
groups

[1] "Colonna C0"   "Colonna C10"  "Colonna C5"   "Colonna C7"   "Colonna C9"  
 [6] "Gardner.A C1" "Gardner.A C3" "Gardner.A C4" "Gardner.E C1" "Gardner.E C7"
[11] "Kedmi C12"    "Kedmi C14"    "Kedmi C6"     "Kedmi C9"     "Littman C1"  
[16] "Littman C14"  "Lyu C11"      "Lyu C2"       "Lyu C6"       "mto C1"      
[21] "mto C10"      "mto C11"      "mto C12"      "mto C13"      "mto C14"     
[26] "mto C15"      "mto C16"      "mto C2"       "mto C3"       "mto C4"      
[31] "mto C5"       "mto C6"       "mto C7"       "mto C8"       "mto C9"      
[36] "TC_all_LN C1" "TC_all_LN C2" "TC_all_LN C3" "TC_all_LN C4" "TC_all_LN C5"
[41] "TC_all_LN C6" "TC_all_LN C7" "Wang C12"     "Wang C13"     "Wang C3"     
[46] "Wang C9"

In [11]:
sources <- c(
    "MLN_RORgt_MHCII_multiome", "TC_all_LN", "Colonna",
    "Kedmi", "Lyu", "Wang",
    "Littman", "Gardner.A", "Gardner.E"
)

pl <- list()
for (i in 1:length(sources)){
    source.curr <- sources[i]
    p.curr <- plot.clusters.highlight.one(
        SRO = sro.i,
        idx1 = colnames(sro.i), idx2 = sro.i$orig.ident == source.curr,
        groups = get.named.vector.sro(sro.i, group.name),
        clusters.col = source.curr,
        labels = F, label.size = 5,
        col = pal[[group.name]], pref.C = F) + guides(colour = guide_legend(override.aes = list(size=5), ncol = 1))
    pl[[i]] <- p.curr
}

ncols <- 3
nrows <- ceiling(length(pl)/ncols)

raster_pdf(file = paste0(pref.p, "UMAP-", group.name, ".pdf"), width = 10*ncols, height = 6*nrows, res = 150)
plot_grid(plotlist = pl, ncol = ncols, align = 'hv')
dev.off()

pdf 
  2

# save changes

In [60]:
# write.csv(sro.i@meta.data, file = paste0(pref, "meta-data.csv"))
# saveRDS(sro.i, paste0(pref, "SRO.rds"))

# final annot

In [12]:
gardner.e.md <- read.csv("../oral-tolerance-Gardner/Seurat/early/meta-data.csv", row.names = 1)
gardner.e.md <- gardner.e.md %>% subset(RNA_snn_res.0.2 %in% c(1, 7, 10))
rownames(gardner.e.md) <- paste0(rownames(gardner.e.md), "_9")
gardner.e.md$paper.annot[is.na(gardner.e.md$paper.annot)] <- 'na'

In [13]:
table(gardner.e.md$paper.annot)


         LTi LTi-like ILC           na       R-cDC1       R-cDC2      R-eTAC1 
        9627           39         1166            3            9          829 
     R-eTAC2      R-eTAC3        R-mDC 
         508          370            2 

In [14]:
sro.i@meta.data[sro.i@meta.data$orig.ident == 'Gardner.E',]$paper.annot <- gardner.e.md[rownames(sro.i@meta.data[sro.i@meta.data$orig.ident == 'Gardner.E',]), ]$paper.annot

In [15]:
kedmi.wang.cl.to.annot <- c(
    'Kedmi C14' = 'LTi-like ILC3',
    'Kedmi C12' = 'Ki67+',
    'Kedmi C6' = 'JC1',
    'Kedmi C9' = 'JC2',
    'Wang C13' = 'LTi-like ILC3',
    'Wang C9' = 'Ki67+',
    'Wang C3' = 'JC1',
    'Wang C12' = 'JC2'
)

In [16]:
sro.i$paper.annot <- ifelse(
    test = sro.i$Cluster.prev %in% names(kedmi.wang.cl.to.annot),
    yes = as.character(kedmi.wang.cl.to.annot[sro.i$Cluster.prev]),
    no = sro.i$paper.annot
)

In [17]:
table(sro.i$paper.annot)[table(sro.i$paper.annot) < 20]


             cDC2A                ILC               ILC2           Mig_DC_1 
                 8                 10                  1                  3 
            R-cDC1             R-cDC2              R-mDC   T cell zone macs 
                 5                 18                 14                  5 
Tingible body macs 
                 2 

In [19]:
annot1.to.annot3 <- c(
    "ILC3" = "ILC3",
    "ILC3p" = "ILC3",
    "Ki67+ TC" = "Ki67+ TC",
    "LTi" = "LTi", 
    "LTi Variation 1" = "LTi Variation 1", "LTi Variation 2" = "LTi Variation 2",
    "LTi Variation 3" = "LTi Variation 3", "LTi Variation 4" = "LTi Variation 4",
    "LTi Variation 5" = "LTi Variation 5", "LTi Variation 6" = "LTi Variation 6",
    "LTi Variation 7" = "LTi Variation 7", "LTi Variation 8" = "LTi Variation 8",
    "LTi-like ILC" = "ILC", "LTi-like ILC3s" = "ILC3",
    'LTi-like ILC3' = 'ILC3',
    "LTi-like R-eTAC" = "LTi-like eTAC",
    "NCR+ ILC3" = "ILC3",
    "Nrg1_Pos" = "Nrg1_Pos",
    "Pro. ILC" = "ILC",
    "Pro. R-eTAC" = "Ki67+ eTAC",
    "Proliferating NCR+ ILC3" = "ILC3",
    "Proliferating TC" = "Ki67+ TC",
    "R-eTAC1" = "eTAC I",
    "R-eTAC2" = "eTAC II", 
    "R-eTAC3" = "eTAC III",
    "TC 1" = "TC I", "TC 2" = "TC II", "TC 3" = "TC III", "TC 4" = "TC IV",
    "TC I" = "TC I", "TC II" = "TC II", "TC III" = "TC III", "TC IV" = "TC IV",
    "eTACs I" = "eTAC I",
    "eTACs II" = "eTAC II",
    'Proliferating eTAC' = 'Ki67+ eTAC',
    "ILC" = "ILC", "ILC1" = "ILC1", "ILC2" = "ILC2",
    "na" = "na",
    "Tingible body macs" = "other",
    "T cell zone macs" = "other",
    "Mig_DC_1" = "other", 
    "R-cDC1" = "other", 
    "R-cDC2" = "other",
    "cDC2A" = "other",
    "R-mDC" = "other"
)


In [20]:
unique(sro.i$paper.annot)[unique(sro.i$paper.annot) %ni% names(annot1.to.annot3)]

[1] "JC1"                   "JC2"                   "Ki67+"                
[4] "early/transitional TC" "RORgt DC I"            "RORgt DC IV"          
[7] "RORgt DC III"          "RORgt DC II"           "Tolerogenic DC"

In [21]:
names(annot1.to.annot3)[names(annot1.to.annot3) %ni% unique(sro.i$paper.annot)]

[1] "Proliferating eTAC"

In [22]:
sro.i$paper.annot3 <- ifelse(
    test = sro.i$paper.annot %in% names(annot1.to.annot3),
    yes = as.character(annot1.to.annot3[sro.i$paper.annot]),
    no = sro.i$paper.annot
)
group.name <- 'paper.annot3'
groups <- sort(unique(sro.i@meta.data[[group.name]]))
groups[groups %ni% names(pal$paper.annot3)]

[1] "early/transitional TC" "eTAC I"                "eTAC II"              
 [4] "eTAC III"              "ILC"                   "ILC1"                 
 [7] "ILC2"                  "ILC3"                  "JC1"                  
[10] "JC2"                   "Ki67+"                 "Ki67+ eTAC"           
[13] "Ki67+ TC"              "LTi"                   "LTi Variation 1"      
[16] "LTi Variation 2"       "LTi Variation 3"       "LTi Variation 4"      
[19] "LTi Variation 5"       "LTi Variation 6"       "LTi Variation 7"      
[22] "LTi Variation 8"       "LTi-like eTAC"         "na"                   
[25] "Nrg1_Pos"              "other"                 "RORgt DC I"           
[28] "RORgt DC II"           "RORgt DC III"          "RORgt DC IV"          
[31] "TC I"                  "TC II"                 "TC III"               
[34] "TC IV"                 "Tolerogenic DC"

In [28]:
pal$paper.annot3 <-c(
    "Proliferating TC" = "#e051bc",
    "Ki67 TC" = "#e051bc",
    "Ki67+ TC" = "#e051bc",
    "Ki67+" = "#e051bc",
    "Ki67+ eTAC" = "#e051bc",
    "early/transitional TC" = "#ccf3ff",
    "TC I" = "#D790FF",
    "TC II" = "#BC23FF",
    "TC III" = "#72418F",
    "TC IV" = "#3A2465",
    "TC II, III, IV" = "#72418F",
    "TCs" = "#BC23FF",
    "RORgt DC I" = "#D790FF",
    "RORgt DC II" = "#BC23FF",
    "RORgt DC III" = "#72418F",
    "RORgt DC IV" = "#3A2465",
    "LTi-like eTAC" = "#ccf3ff",
    "Proliferating eTAC" = "#e051bc",
    "RORgt+_DC_like" = '#D790FF',
    "RORgt+_eTAC" = '#72418F',
    "JC1" = "#D790FF",
    "JC2" = "#BC23FF",
    "eTAC I" = "#D790FF",
    "eTAC II" = "#BC23FF",
    "eTAC III" = "#72418F",
    "JC"= "#BC23FF",
    "ILC1" ='#17e81b',
    "ILC2" ='#77d9ae',
    "ILC3p" = "#7daedb",
    "Proliferating NCR+ ILC3" = "#0976de",
    "ILC3" = "#14a38e",
    "NCR+ ILC3" = "#003a85",
    "LTi-like ILC3s" = "#14a38e",
    "LTi-like ILC3" = "#14a38e",
    "LTi Variation 1" = "#17e81b",
    "LTi Variation 2" = "#77d9ae",
    "LTi Variation 3" = "#14a38e",
    "LTi Variation 4" = "#05a15c",
    "LTi Variation 5" = "#4e7506",
    "LTi Variation 6" = "#0d6930",
    "LTi Variation 7" = "#405933",
    "LTi Variation 8" = "#0d3b0e",
    'LTi' = "#14a38e",
    "ILC" ='#14a38e',
    "Nrg1_Pos" = '#D790FF',
    "Tolerogenic DC" = '#72418F',
    "mDC" = '#FAD09F',
    "pDC" = '#e67109',
    "tDC" = '#ffc5af',
    "migDC" = 'orange',
    "cDC1" = 'orange',
    "cDC2" = '#e60e0e',
    "cDC2A" = '#e60e0e',
    "cDC2_A" = '#e60e0e',
    "cDC2_B" = 'pink',
    "Tingible body macs" =  '#0077ff',
    "T cell zone macs" = '#489de8',
    "na" = 'black',
    'other' = 'yellow'
    # 'other' = '#636363'
)

In [24]:
sro.i$paper.annot3 <- factor(sro.i$paper.annot3, levels = names(pal$paper.annot3))
sources <- c(
    "MLN_RORgt_MHCII_multiome", "TC_all_LN", "Colonna",
    "Kedmi", "Lyu", "Wang",
    "Littman", "Gardner.A", "Gardner.E"
)

In [29]:
pl <- list()
for (i in 1:length(sources)){
    source.curr <- sources[i]
    p.curr <- plot.clusters.highlight.one(
        SRO = sro.i,
        idx1 = colnames(sro.i), idx2 = sro.i$orig.ident == source.curr,
        groups = get.named.vector.sro(sro.i, group.name),
        clusters.col = source.curr,
        labels = F, label.size = 5,
        col = pal[[group.name]], pref.C = F) + guides(colour = guide_legend(override.aes = list(size=5), ncol = 1))
    pl[[i]] <- p.curr
}

ncols <- 3
nrows <- ceiling(length(pl)/ncols)

raster_pdf(file = paste0(pref.p, "UMAP-", group.name, ".pdf"), width = 10*ncols, height = 6*nrows, res = 150)
plot_grid(plotlist = pl, ncol = ncols, align = 'hv')
dev.off()

pdf 
  2